In [73]:
import pandas as pd
import numpy as np

filename = '../data/raw/supply_chain_raw.csv'
pd.set_option('display.max_columns', None)

### Extract

In [13]:
raw_df = pd.read_csv(filename, encoding='cp1252')
raw_df.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [14]:
raw_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Type                           180519 non-null  str    
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Sales per customer             180519 non-null  float64
 5   Delivery Status                180519 non-null  str    
 6   Late_delivery_risk             180519 non-null  int64  
 7   Category Id                    180519 non-null  int64  
 8   Category Name                  180519 non-null  str    
 9   Customer City                  180519 non-null  str    
 10  Customer Country               180519 non-null  str    
 11  Customer Email                 180519 non-null  str    
 12  Customer Fname                 180519 non

In [64]:
for col in raw_df.select_dtypes(include=['object', 'string']).columns: 
    
    mask_cortos = raw_df[col].str.strip().str.len() <= 10
    
    # Solo mostramos si realmente encontramos algo
    if mask_cortos.any():
        print(raw_df.loc[mask_cortos, col].value_counts().head(5))
        print("-" * 30)

Type
DEBIT       69295
TRANSFER    49883
PAYMENT     41725
CASH        19616
Name: count, dtype: int64
------------------------------
Category Name
Cleats        24551
Fishing       17325
Golf Balls     1475
Trade-In        974
Hockey          614
Name: count, dtype: int64
------------------------------
Customer City
Caguas      66770
Chicago      3885
Brooklyn     3412
New York     1816
Bronx        1500
Name: count, dtype: int64
------------------------------
Customer Country
EE. UU.    111146
Name: count, dtype: int64
------------------------------
Customer Email
XXXXXXXXX    180519
Name: count, dtype: int64
------------------------------
Customer Fname
Mary       65150
James       1835
Robert      1759
Michael     1680
David       1625
Name: count, dtype: int64
------------------------------
Customer Lname
Smith       64104
Johnson       989
Brown         909
Williams      869
Jones         859
Name: count, dtype: int64
------------------------------
Customer Password
XXXXXXXXX    

In [15]:
clean_df = raw_df.copy()

### Transform

### 2.

In [16]:
clean_df.columns = clean_df.columns.str.strip().str.lower().str.replace(' ', '_')
clean_df = clean_df[clean_df.columns.difference(['product_description', 'customer_email', 'customer_password'])]

### 3.

In [70]:
for col in clean_df.select_dtypes(include=['object', 'string']).columns:
    clean_df[col] = clean_df[col].str.strip().str.title()

for col in ['customer_state', 'type', 'order_status', 'market']:
    clean_df[col] = clean_df[col].str.upper()

fix_dict = {
    'Ee. Uu.': 'EE. UU.'
}

clean_df['customer_country'] = clean_df['customer_country'].replace(fix_dict)

In [74]:
clean_df.head()

,benefit_per_order,category_id,category_name,customer_city,customer_country,customer_fname,customer_id,customer_lname,customer_segment,customer_state,customer_street,customer_zipcode,days_for_shipment_(scheduled),days_for_shipping_(real),delivery_status,department_id,department_name,late_delivery_risk,latitude,longitude,market,order_city,order_country,order_customer_id,order_date_(dateorders),order_id,order_item_cardprod_id,order_item_discount,order_item_discount_rate,order_item_id,order_item_product_price,order_item_profit_ratio,order_item_quantity,order_item_total,order_profit_per_order,order_region,order_state,order_status,order_zipcode,product_card_id,product_category_id,product_image,product_name,product_price,product_status,sales,sales_per_customer,shipping_date_(dateorders),shipping_mode,type
0,91.250000,73,Sporting Goods,Caguas,Puerto Rico,Cally,20755,Holloway,Consumer,PR,5365 Noble Nectar Island,725.0,4,3,Advance Shipping,2,Fitness,0,18.251453,-66.037056,PACIFIC ASIA,Bekasi,Indonesia,20755,1/31/2018 22:56,77202,1360,13.110000,0.04,180517,327.75,0.29,1,314.640015,91.250000,Southeast Asia,Java Occidental,COMPLETE,NaN,1360,73,Http://Images.Acmesports.Sports/Smart+Watch,Smart Watch,327.75,0,327.75,314.640015,2/3/2018 22:56,Standard Class,DEBIT
1,-249.089996,73,Sporting Goods,Caguas,Puerto Rico,Irene,19492,Luna,Consumer,PR,2679 Rustic Loop,725.0,4,5,Late Delivery,2,Fitness,1,18.279451,-66.037064,PACIFIC ASIA,Bikaner,India,19492,1/13/2018 12:27,75939,1360,16.389999,0.05,179254,327.75,-0.80,1,311.359985,-249.089996,South Asia,Rajastán,PENDING,NaN,1360,73,Http://Images.Acmesports.Sports/Smart+Watch,Smart Watch,327.75,0,327.75,311.359985,1/18/2018 12:27,Standard Class,TRANSFER
2,-247.779999,73,Sporting Goods,San Jose,EE. UU.,Gillian,19491,Maldonado,Consumer,CA,8510 Round Bear Gate,95125.0,4,4,Shipping On Time,2,Fitness,0,37.292233,-121.881279,PACIFIC ASIA,Bikaner,India,19491,1/13/2018 12:06,75938,1360,18.030001,0.06,179253,327.75,-0.80,1,309.720001,-247.779999,South Asia,Rajastán,CLOSED,NaN,1360,73,Http://Images.Acmesports.Sports/Smart+Watch,Smart Watch,327.75,0,327.75,309.720001,1/17/2018 12:06,Standard Class,CASH
3,22.860001,73,Sporting Goods,Los Angeles,EE. UU.,Tana,19490,Tate,Home Office,CA,3200 Amber Bend,90027.0,4,3,Advance Shipping,2,Fitness,0,34.125946,-118.291016,PACIFIC ASIA,Townsville,Australia,19490,1/13/2018 11:45,75937,1360,22.940001,0.07,179252,327.75,0.08,1,304.809998,22.860001,Oceania,Queensland,COMPLETE,NaN,1360,73,Http://Images.Acmesports.Sports/Smart+Watch,Smart Watch,327.75,0,327.75,304.809998,1/16/2018 11:45,Standard Class,DEBIT
4,134.210007,73,Sporting Goods,Caguas,Puerto Rico,Orli,19489,Hendricks,Corporate,PR,8671 Iron Anchor Corners,725.0,4,2,Advance Shipping,2,Fitness,0,18.253769,-66.037048,PACIFIC ASIA,Townsville,Australia,19489,1/13/2018 11:24,75936,1360,29.500000,0.09,179251,327.75,0.45,1,298.250000,134.210007,Oceania,Queensland,PENDING_PAYMENT,NaN,1360,73,Http://Images.Acmesports.Sports/Smart+Watch,Smart Watch,327.75,0,327.75,298.250000,1/15/2018 11:24,Standard Class,PAYMENT


In [72]:
clean_df['customer_country'].unique()

<StringArray>
['Puerto Rico', 'EE. UU.']
Length: 2, dtype: str

In [76]:
clean_df.filter(like='customer').head()

,customer_city,customer_country,customer_fname,customer_id,customer_lname,customer_segment,customer_state,customer_street,customer_zipcode,order_customer_id,sales_per_customer
0,Caguas,Puerto Rico,Cally,20755,Holloway,Consumer,PR,5365 Noble Nectar Island,725.0,20755,314.640015
1,Caguas,Puerto Rico,Irene,19492,Luna,Consumer,PR,2679 Rustic Loop,725.0,19492,311.359985
2,San Jose,EE. UU.,Gillian,19491,Maldonado,Consumer,CA,8510 Round Bear Gate,95125.0,19491,309.720001
3,Los Angeles,EE. UU.,Tana,19490,Tate,Home Office,CA,3200 Amber Bend,90027.0,19490,304.809998
4,Caguas,Puerto Rico,Orli,19489,Hendricks,Corporate,PR,8671 Iron Anchor Corners,725.0,19489,298.250000
